<a></a>
# ResNet - Skin Lesions Image Classification with ISIC 2019

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


<a></a>
## 1. Initializations and Imports

In [ ]:
# Train fcc
second = False
epochs = 60 #sse mudo aqui tenho que mudar fcc o lr no_epochs=50
# freeze = false - means initialization but not freeze
freeze = False


### Filepath saved weights 
random = False # true = no weights loaded from the previous task

# Path to previously trained model
filepath = '/content/drive/MyDrive/Tese/fold4_exp_rot_newarq.h5'
# Path to save new model
filepath_fcc = '/content/drive/MyDrive/Bolsa-IMP/fold4/rot/fold4_eval_rot_imag_newarq_final.h5'


#----------------------------------
### Network 
weight_resnet = False #true =imagenet
res = 'old'
# res = 'new'
## Resnet50
rede = 'resnet50' 
h1 = 256
h2 = 128

#----------------------------------
### Evaluation method
aval = 'fcc'
extra_mlp = False # true if we want fcc after the gap and before softmax
h1_fcc = 512
h2_fcc = 128

# aval = 'svm'
param_c = 100
param_g = 0.1

#----------------------------------
### Feature Space
fspace = 'gap'  #2048 / 64
# fspace = 'h2' #128 / 32
# fspace = 'h1' #256 / 64



In [ ]:
if second == False:
    !unzip /content/drive/MyDrive/Tese/ISIC_2019_pre-processed_fold4.zip -d /content

In [ ]:
if second == False:
    !unzip /content/drive/MyDrive/Tese/ISIC_2019_pre-processed.zip -d /content

In [ ]:
import csv
import pandas as pd
import numpy as np
import os,sys
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from collections import Counter

sys.path.append('/content/drive/MyDrive/Tese/')

from visuals import plot_grouped_2bars
import math
import shutil
from sklearn.utils import shuffle

import glob

import os, os.path
from ast import literal_eval
from make_dir_versao_writedisco import *

import cv2

from PIL import Image


import keras
from tensorflow.keras.layers import Input, Dense, Activation,Flatten, Convolution2D, Conv2D, MaxPooling2D, UpSampling2D, GlobalAveragePooling2D, concatenate, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint

from tensorflow.keras import layers, losses

import tensorflow as tf
from tensorflow import keras
from tensorflow.python.keras import preprocessing
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adadelta, RMSprop,SGD,Adam
#from tensorflow.python.keras.preprocessing.image import image_dataset_from_directory

from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score,confusion_matrix

from mpl_toolkits.mplot3d import Axes3D

from tensorflow.keras.models import Sequential, save_model, load_model

from sklearn.svm import SVC
from sklearn import svm
from sklearn.model_selection import GridSearchCV , StratifiedShuffleSplit
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.metrics import classification_report

import seaborn as sns 

import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from sklearn.utils import class_weight
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from PIL import Image
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau,TensorBoard
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

NUM_TRAIN_SAMPLES = 17731
NUM_VAL_SAMPLES = 7600
BATCH_SIZE = 32


import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.applications import VGG16

import numpy as np
import tensorflow as tf
from tensorflow import keras

# Display
from IPython.display import Image, display
import matplotlib.pyplot as plt
import matplotlib.cm as cm

<a></a>
## 2. Online Data Augmentation

In [ ]:
from tensorflow.python.keras.applications.resnet import ResNet50, preprocess_input
datagen = ImageDataGenerator(
        preprocessing_function=preprocess_input,
        rotation_range=90,
        #width_shift_range=0.2,
        #height_shift_range=0.2,
        #rescale=1./255,
        #shear_range=0.2,
        #brightness_range=[0.2,1],
        vertical_flip = True,
        horizontal_flip=True)

datagenV = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    #rescale=1./255
    )

seed = 1
train_generator = datagen.flow_from_directory(
    directory = '/content/ISIC_2019_pre-processed_fold4/train',
    target_size = (224,224),
    color_mode = 'rgb',
    shuffle = True,
    batch_size = BATCH_SIZE,
    class_mode = 'categorical',
    seed = seed)
seed = 2
validation_generator = datagenV.flow_from_directory(
    directory = '/content/ISIC_2019_pre-processed_fold4/valid',
    target_size = (224,224),
    color_mode = 'rgb',
    shuffle = True,
    batch_size = BATCH_SIZE,
    class_mode = 'categorical',
    seed = seed)
seed = 3
trainPredict_generator = datagen.flow_from_directory(
    directory = '/content/ISIC_2019_pre-processed_fold4/train',
    target_size = (224,224),
    color_mode = 'rgb',
    shuffle = False,
    batch_size = BATCH_SIZE,
    class_mode = 'categorical',
    seed = seed)
seed = 4
validationPredict_generator = datagenV.flow_from_directory(
    directory =  '/content/ISIC_2019_pre-processed_fold4/valid',
    target_size = (224,224),
    color_mode = 'rgb',
    shuffle = False,
    batch_size = BATCH_SIZE,
    class_mode = 'categorical',
    seed = seed)


Found 17731 images belonging to 8 classes.
Found 7600 images belonging to 8 classes.
Found 17731 images belonging to 8 classes.
Found 7600 images belonging to 8 classes.


<a></a>
## 3.Build ResNet Architecture

In [ ]:
from tensorflow.keras.applications.resnet import ResNet50
MODEL = ResNet50(include_top=False, weights= 'imagenet', input_tensor=None, input_shape=(224,224,3), pooling=None)
# MODEL.summary()

94781440/94765736 [==============================] - 0s 0us/step


In [ ]:
from tensorflow.keras.layers import GlobalAveragePooling2D

from tensorflow.keras.applications.resnet import ResNet50
inputs = tf.keras.Input(shape=(224, 224, 3))
x = ResNet50(input_tensor=inputs,include_top=False, weights= 'imagenet', input_shape=(224,224,3), pooling=None)
x = layers.GlobalAveragePooling2D()(x.output)
x = layers.Dropout(0.5)(x)
x = layers.Dense(4, activation='softmax')(x)
model = keras.Model(inputs, x)
model.summary()

if random == False:
  model.load_weights(filepath)
  print('loaded weights from rot')

full_model_gap = Model(inputs=model.input, outputs=model.layers[-3].output)
# full_model_gap.summary()

# x = layers.GlobalAveragePooling2D()(full_model_gap.output)
x = layers.Dropout(0.5)(full_model_gap.output)
x = layers.Dense(8, activation='softmax')(x)
model = keras.Model(full_model_gap.inputs, x)
model.summary()

optimizer = tf.keras.optimizers.Adam(lr=0.00001, beta_1=0.9, beta_2=0.999, decay=0.0)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])



In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True, show_dtype=True)


<a></a>
## 3. Fit

In [ ]:
def lr_scheduler(epoch, lr):
    decay_rate = 0.3
    if epoch == 2 or epoch == 4 or epoch ==7 or epoch == 15  or epoch == 20:
        return lr*decay_rate 
    return lr

saveWeights = ModelCheckpoint(filepath_fcc, save_best_only=True, monitor='val_loss', mode='min')
learningRate = ReduceLROnPlateau(monitor='val_loss', factor=0.75, patience=5, verbose=1,  mode='min')
#tensBoard = TensorBoard(log_dir='./logs', histogram_freq=0, batch_size=BATCH_SIZE, write_graph=True, write_grads=True, write_images=True, update_freq='epoch')



class_weights = {0: NUM_TRAIN_SAMPLES/3165,
                1: NUM_TRAIN_SAMPLES/9012,
                2: NUM_TRAIN_SAMPLES/2326,
                3: NUM_TRAIN_SAMPLES/607,
                4: NUM_TRAIN_SAMPLES/1837,
                5: NUM_TRAIN_SAMPLES/167,
                6: NUM_TRAIN_SAMPLES/177,
                7: NUM_TRAIN_SAMPLES/440}


# model.load_weights(filepath_fcc)
history = model.fit_generator(train_generator,
                    #steps_per_epoch = NUM_TRAIN_SAMPLES // BATCH_SIZE,
                    epochs = 50, verbose =1, 
                    validation_data = validation_generator,
                    shuffle = True,
                    workers = 4, 
                    #callbacks =[LearningRateScheduler(lr_scheduler, verbose=1),saveWeights,learningRate, tensBoard],
                    callbacks =[saveWeights,learningRate],
                    #validation_steps = NUM_VAL_SAMPLES // BATCH_SIZE,
                    class_weight = class_weights)
    


<a></a>
## 4. Load the Trained model

In [ ]:
from tensorflow.keras.models import model_from_json

# load weights into new model
model.load_weights(filepath_fcc)


In [ ]:

def get_img_array(img_path, size):
    # `img` is a PIL image of size 299x299
    img = keras.preprocessing.image.load_img(img_path, target_size=size)
    # `array` is a float32 Numpy array of shape (299, 299, 3)
    array = keras.preprocessing.image.img_to_array(img)
    # We add a dimension to transform our array into a "batch"
    # of size (1, 299, 299, 3)
    array = np.expand_dims(array, axis=0)
    return array


def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    # First, we create a model that maps the input image to the activations
    # of the last conv layer as well as the output predictions
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )

    # Then, we compute the gradient of the top predicted class for our input image
    # with respect to the activations of the last conv layer
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    # This is the gradient of the output neuron (top predicted or chosen)
    # with regard to the output feature map of the last conv layer
    grads = tape.gradient(class_channel, last_conv_layer_output)

    # This is a vector where each entry is the mean intensity of the gradient
    # over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # We multiply each channel in the feature map array
    # by "how important this channel is" with regard to the top predicted class
    # then sum all the channels to obtain the heatmap class activation
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # For visualization purpose, we will also normalize the heatmap between 0 & 1
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

In [ ]:
def save_and_display_gradcam(img_path, heatmap, cam_path="cam.jpg", alpha=0.4):
    # Load the original image
    img = keras.preprocessing.image.load_img(img_path)
    img = keras.preprocessing.image.img_to_array(img)

    # Rescale heatmap to a range 0-255
    heatmap = np.uint8(255 * heatmap)

    # Use jet colormap to colorize heatmap
    jet = cm.get_cmap("jet")

    # Use RGB values of the colormap
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    # Create an image with RGB colorized heatmap
    jet_heatmap = keras.preprocessing.image.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
    jet_heatmap = keras.preprocessing.image.img_to_array(jet_heatmap)

    # Superimpose the heatmap on original image
    superimposed_img = jet_heatmap * alpha + img
    superimposed_img = keras.preprocessing.image.array_to_img(superimposed_img)

    # Save the superimposed image
    superimposed_img.save(cam_path)

    # Display Grad CAM
    display(Image(cam_path))



In [ ]:

gen_filepaths = validation_generator.filepaths
print(gen_filepaths)

import random
K_NUM=200

# # chooses k random filepaths from the filepath list:
gen_14 = random.choices(gen_filepaths, k=K_NUM)
np.save('/content/drive/MyDrive/Tese/gen_14_2.npy', gen_14)

gen_14 = np.load('/content/drive/MyDrive/Tese/gen_14_2.npy')
img_size = (224,224)

['/content/ISIC_2019_pre-processed_fold4/valid/0/ISIC_0000013.jpg', '/content/ISIC_2019_pre-processed_fold4/valid/0/ISIC_0000022_downsampled.jpg', '/content/ISIC_2019_pre-processed_fold4/valid/0/ISIC_0000031_downsampled.jpg', '/content/ISIC_2019_pre-processed_fold4/valid/0/ISIC_0000056_downsampled.jpg', '/content/ISIC_2019_pre-processed_fold4/valid/0/ISIC_0000147_downsampled.jpg', '/content/ISIC_2019_pre-processed_fold4/valid/0/ISIC_0000151_downsampled.jpg', '/content/ISIC_2019_pre-processed_fold4/valid/0/ISIC_0000152_downsampled.jpg', '/content/ISIC_2019_pre-processed_fold4/valid/0/ISIC_0000155_downsampled.jpg', '/content/ISIC_2019_pre-processed_fold4/valid/0/ISIC_0000156_downsampled.jpg', '/content/ISIC_2019_pre-processed_fold4/valid/0/ISIC_0000160.jpg', '/content/ISIC_2019_pre-processed_fold4/valid/0/ISIC_0000161.jpg', '/content/ISIC_2019_pre-processed_fold4/valid/0/ISIC_0000165.jpg', '/content/ISIC_2019_pre-processed_fold4/valid/0/ISIC_0000169.jpg', '/content/ISIC_2019_pre-processe

<a></a>
## 5. GradCAM

In [ ]:
COUNT=0
for img_path in gen_14:
  # Prepare image
  COUNT=COUNT+1
  img_array = preprocess_input(get_img_array(img_path, size=img_size))

  print(COUNT)
  # Print what the top predicted class is
  preds = model.predict(img_array)
  # print("Predicted:", np.argmax(preds, axis=1))


  last_conv_layer_name = 'conv5_block3_out'
  print('\n\n \n\nNew image:')
  # Generate class activation heatmap
  print('Predicted softmax vector:',preds)
  # print("Predicted:", np.argmax(preds, axis=1))
  
  ## codigo para retirar label do nome #####################################
  label = []
  # print('label: {}| predicted: {}'.format(label, np.argmax(preds, axis=1)))
  print('\n Original Image label: {}| predicted: {}'.format(img_path[45], np.argmax(preds, axis=1)))
  if int(img_path[45]) == int(np.argmax(preds, axis=1)):
    print(' ---->TRUE <-----')
  image = plt.imread(img_path)
  plt.imshow(image)

  # print('\n Heatmap:')
  heatmap = make_gradcam_heatmap(img_array, model, last_conv_layer_name)

  # Display heatmap
  plt.matshow(heatmap)
  plt.show()

  print('\nAll together:')
  save_and_display_gradcam(img_path, heatmap)